## 准备数据

In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, optimizers, datasets

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # or any {'0', '1', '2'}

def mnist_dataset():
    (x, y), (x_test, y_test) = datasets.mnist.load_data()
    #normalize
    x = x/255.0
    x_test = x_test/255.0
    
    return (x, y), (x_test, y_test)

2024-03-08 16:37:09.256784: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-08 16:37:09.280546: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-08 16:37:09.638165: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
print(list(zip([1, 2, 3, 4], ['a', 'b', 'c', 'd'])))

[(1, 'a'), (2, 'b'), (3, 'c'), (4, 'd')]


## 建立模型

In [3]:
class myModel:
    def __init__(self):
        ####################
        '''声明模型对应的参数'''
        ####################
        # 输入图片大小为28*28，隐藏层大小为128，输出具有10个类型
        self.W1 = tf.Variable(shape=[28*28, 128],
                               dtype=tf.float32,
                               initial_value=tf.random.uniform(shape=[28*28, 128],
                                                               minval=-.1,
                                                               maxval=.1))
        self.b1 = tf.Variable(shape=[128],
                             dtype=tf.float32,
                             initial_value=tf.zeros(128))
        
        self.W2 = tf.Variable(shape=[128, 10],
                               dtype=tf.float32,
                               initial_value=tf.random.uniform(shape=[128, 10],
                                                               minval=-.1,
                                                               maxval=.1))
        self.b2 = tf.Variable(shape=[10],
                             dtype=tf.float32,
                             initial_value=tf.zeros(10))
        
    def __call__(self, x):
        ####################
        '''实现模型函数体，返回未归一化的logits'''
        ####################
        # 将x展开为1-d向量 shape=(N, 28*28)
        flattened_x = tf.reshape(x, shape=[-1, 28*28])
        # x经过一层神经元 shape=(N, 128)
        raw_hidden_value = tf.matmul(flattened_x, self.W1)+self.b1
        # raw_hidden_value经过激活函数，这里使用tanh shape=(N, 128)
        hidden_value = tf.tanh(raw_hidden_value)
        # hidden_value经过输出层神经元，输出logits逻辑值 shape=(N, 10)
        logits = tf.matmul(hidden_value, self.W2)+self.b2
        return logits
        
model = myModel()

optimizer = optimizers.Adam()

2024-03-08 16:37:09.996864: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-03-08 16:37:10.013957: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


## 计算 loss

In [4]:
@tf.function
def compute_loss(logits, labels):
    return tf.reduce_mean(
        tf.nn.sparse_softmax_cross_entropy_with_logits(
            logits=logits, labels=labels))

@tf.function
def compute_accuracy(logits, labels):
    predictions = tf.argmax(logits, axis=1)
    return tf.reduce_mean(tf.cast(tf.equal(predictions, labels), tf.float32))

@tf.function
def train_one_step(model, optimizer, x, y):
    with tf.GradientTape() as tape:
        logits = model(x)
        loss = compute_loss(logits, y)

    # compute gradient
    trainable_vars = [model.W1, model.W2, model.b1, model.b2]
    grads = tape.gradient(loss, trainable_vars)
    for g, v in zip(grads, trainable_vars):
        v.assign_sub(0.01*g)

    accuracy = compute_accuracy(logits, y)

    # loss and accuracy is scalar tensor
    return loss, accuracy

@tf.function
def test(model, x, y):
    logits = model(x)
    loss = compute_loss(logits, y)
    accuracy = compute_accuracy(logits, y)
    return loss, accuracy

## 实际训练

In [5]:
train_data, test_data = mnist_dataset()
for epoch in range(50):
    loss, accuracy = train_one_step(model, optimizer, 
                                    tf.constant(train_data[0], dtype=tf.float32), 
                                    tf.constant(train_data[1], dtype=tf.int64))
    print('epoch', epoch, ': loss', loss.numpy(), '; accuracy', accuracy.numpy())
loss, accuracy = test(model, 
                      tf.constant(test_data[0], dtype=tf.float32), 
                      tf.constant(test_data[1], dtype=tf.int64))

print('test loss', loss.numpy(), '; accuracy', accuracy.numpy())

epoch 0 : loss 2.3103702 ; accuracy 0.14275
epoch 1 : loss 2.3028984 ; accuracy 0.14665
epoch 2 : loss 2.2955437 ; accuracy 0.1518
epoch 3 : loss 2.2882996 ; accuracy 0.15626666
epoch 4 : loss 2.2811599 ; accuracy 0.16088334
epoch 5 : loss 2.2741199 ; accuracy 0.16613333
epoch 6 : loss 2.2671735 ; accuracy 0.17123333
epoch 7 : loss 2.2603166 ; accuracy 0.1763
epoch 8 : loss 2.2535448 ; accuracy 0.18131667
epoch 9 : loss 2.2468536 ; accuracy 0.1872
epoch 10 : loss 2.24024 ; accuracy 0.19248334
epoch 11 : loss 2.2336998 ; accuracy 0.19748333
epoch 12 : loss 2.2272296 ; accuracy 0.20198333
epoch 13 : loss 2.2208273 ; accuracy 0.20761667
epoch 14 : loss 2.2144887 ; accuracy 0.21315
epoch 15 : loss 2.2082121 ; accuracy 0.2188
epoch 16 : loss 2.201995 ; accuracy 0.22468333
epoch 17 : loss 2.195834 ; accuracy 0.23118334
epoch 18 : loss 2.1897275 ; accuracy 0.23731667
epoch 19 : loss 2.1836736 ; accuracy 0.24453333
epoch 20 : loss 2.17767 ; accuracy 0.2522
epoch 21 : loss 2.171715 ; accuracy 0